In [1]:
''' Football Player Performance - Data Preprocessing

This notebook handles data cleaning, feature engineering, and preparation for the 5 main objectives:

1. **Player Rating Prediction**
2. **Goals/Assists Prediction** 
3. **Player Potential Evaluation**
4. **Training Plan Recommendations**
5. **Tactics/Formation Recommendations**

## Data Sources:
- Player statistics from 8 major football leagues
- 12 different CSV files per team covering various performance aspects
- Historical match data and individual player metrics
'''

' Football Player Performance - Data Preprocessing\n\nThis notebook handles data cleaning, feature engineering, and preparation for the 5 main objectives:\n\n1. **Player Rating Prediction**\n2. **Goals/Assists Prediction** \n3. **Player Potential Evaluation**\n4. **Training Plan Recommendations**\n5. **Tactics/Formation Recommendations**\n\n## Data Sources:\n- Player statistics from 8 major football leagues\n- 12 different CSV files per team covering various performance aspects\n- Historical match data and individual player metrics\n'

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
warnings.filterwarnings('ignore')

# Configuration
base_path = "archive (3)"
leagues = ["EPL", "La Liga", "Bundesliga", "Serie A", "Ligue 1", "Eredivise", "Segunda Division", "Brasil Serie A"]

print("Data Preprocessing Module Initialized")
print(f"Working with {len(leagues)} leagues")


Data Preprocessing Module Initialized
Working with 8 leagues


In [3]:
## 1. Data Loading Functions


In [4]:
def load_all_data(file_type="all"):
    """
    Load data from all teams across all leagues
    
    Parameters:
    file_type: str - specific file type to load, or "all" for all files
    
    Returns:
    dict: Dictionary with dataframes organized by file type
    """
    
    data_collection = {}
    
    file_types = [
        "players.csv", "matches.csv", "playing_time.csv", "shooting.csv",
        "passing.csv", "defensive_actions.csv", "possession.csv",
        "goalkeepers.csv", "advanced_goalkeeping.csv", "g_e_s_creation.csv",
        "pass_types.csv", "miscellaneous_stats.csv"
    ]
    
    if file_type != "all":
        file_types = [file_type] if file_type in file_types else []
    
    for file_name in file_types:
        print(f"Loading {file_name}...")
        all_data = []
        
        for league in leagues:
            league_path = os.path.join(base_path, league)
            if os.path.exists(league_path):
                team_folders = [f for f in os.listdir(league_path) 
                              if os.path.isdir(os.path.join(league_path, f))]
                
                for team in team_folders:
                    file_path = os.path.join(league_path, team, file_name)
                    if os.path.exists(file_path):
                        try:
                            df = pd.read_csv(file_path)
                            df['League'] = league
                            df['Team'] = team
                            df['Data_Source'] = f"{league}_{team}"
                            all_data.append(df)
                        except Exception as e:
                            print(f"Error loading {file_path}: {e}")
        
        if all_data:
            combined_df = pd.concat(all_data, ignore_index=True)
            data_collection[file_name.replace('.csv', '')] = combined_df
            print(f"  ✓ Loaded {len(combined_df)} records from {len(all_data)} teams")
        else:
            print(f"  ✗ No data found for {file_name}")
    
    return data_collection

# Load sample data to understand structure
print("=== LOADING SAMPLE DATA ===")
sample_data = load_all_data("players.csv")
if sample_data:
    print(f"Sample data loaded: {list(sample_data.keys())}")
    for key, df in sample_data.items():
        print(f"{key}: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        break


=== LOADING SAMPLE DATA ===
Loading players.csv...
  ✓ Loaded 5942 records from 157 teams
Sample data loaded: ['players']
players: (5942, 38)
Columns: ['Player', 'Nation', 'Pos', 'Age', 'Playing Time MP', 'Playing Time Starts', 'Playing Time Min', 'Playing Time 90s', 'Performance Gls', 'Performance Ast', 'Performance G+A', 'Performance G-PK', 'Performance PK', 'Performance PKatt', 'Performance CrdY', 'Performance CrdR', 'Expected xG', 'Expected npxG', 'Expected xAG', 'Expected npxG+xAG', 'Progression PrgC', 'Progression PrgP', 'Progression PrgR', 'Per 90 Minutes Gls', 'Per 90 Minutes Ast', 'Per 90 Minutes G+A', 'Per 90 Minutes G-PK', 'Per 90 Minutes G+A-PK', 'Per 90 Minutes xG', 'Per 90 Minutes xAG', 'Per 90 Minutes xG+xAG', 'Per 90 Minutes npxG', 'Per 90 Minutes npxG+xAG', 'Matches', 'League', 'Team', 'Data_Source', 'MP']


In [5]:
# Example usage with error handling
print("=== TESTING PREPROCESSING PIPELINE ===")

# Check if we have data loaded
if 'sample_data' in globals() and sample_data:
    # Get the first dataset
    data_key = list(sample_data.keys())[0]
    raw_df = sample_data[data_key]
    
    print(f"\nUsing {data_key} dataset for testing...")
    print(f"Original shape: {raw_df.shape}")
    print(f"Columns: {list(raw_df.columns[:10])}...")  # Show first 10 columns
    
    # Simple preprocessing test
    print("\n🧹 Testing basic data cleaning...")
    
    # Check for missing values
    missing_counts = raw_df.isnull().sum()
    print(f"Missing values found in {(missing_counts > 0).sum()} columns")
    
    # Check data types
    print(f"Data types: {raw_df.dtypes.value_counts().to_dict()}")
    
    # Basic cleaning
    cleaned_df = raw_df.copy()
    
    # Remove completely empty columns
    cleaned_df = cleaned_df.dropna(axis=1, how='all')
    
    # Fill missing values with median for numeric, mode for categorical
    numeric_cols = cleaned_df.select_dtypes(include=[np.number]).columns
    categorical_cols = cleaned_df.select_dtypes(include=['object']).columns
    
    print(f"\nFound {len(numeric_cols)} numeric and {len(categorical_cols)} categorical columns")
    
    # Simple imputation
    for col in numeric_cols:
        if cleaned_df[col].isnull().sum() > 0:
            cleaned_df[col] = cleaned_df[col].fillna(cleaned_df[col].median())
    
    for col in categorical_cols:
        if cleaned_df[col].isnull().sum() > 0:
            cleaned_df[col] = cleaned_df[col].fillna('Unknown')
    
    print(f"✅ Basic cleaning completed!")
    print(f"Shape after cleaning: {cleaned_df.shape}")
    print(f"Missing values remaining: {cleaned_df.isnull().sum().sum()}")
    
else:
    print("⚠️ No sample data available. Please run the previous cells first.")
    print("Make sure to execute Cell 3 which loads the sample data.")


=== TESTING PREPROCESSING PIPELINE ===

Using players dataset for testing...
Original shape: (5942, 38)
Columns: ['Player', 'Nation', 'Pos', 'Age', 'Playing Time MP', 'Playing Time Starts', 'Playing Time Min', 'Playing Time 90s', 'Performance Gls', 'Performance Ast']...

🧹 Testing basic data cleaning...
Missing values found in 2 columns
Data types: {dtype('float64'): 29, dtype('O'): 8, dtype('int64'): 1}

Found 30 numeric and 8 categorical columns
✅ Basic cleaning completed!
Shape after cleaning: (5942, 38)
Missing values remaining: 0


In [6]:
def clean_player_data(df):
    """
    Comprehensive data cleaning for player datasets
    
    Parameters:
    df: pandas DataFrame - raw player data
    
    Returns:
    df_clean: pandas DataFrame - cleaned data
    """
    df_clean = df.copy()
    
    print(f"Original data shape: {df_clean.shape}")
    
    # 1. Remove completely empty rows and columns
    df_clean = df_clean.dropna(how='all')  # Remove rows with all NaN
    df_clean = df_clean.dropna(axis=1, how='all')  # Remove columns with all NaN
    
    # 2. Handle duplicate records
    initial_rows = len(df_clean)
    df_clean = df_clean.drop_duplicates()
    duplicates_removed = initial_rows - len(df_clean)
    if duplicates_removed > 0:
        print(f"Removed {duplicates_removed} duplicate rows")
    
    # 3. Clean column names (remove special characters, spaces)
    df_clean.columns = df_clean.columns.str.strip()  # Remove whitespace
    df_clean.columns = df_clean.columns.str.replace(' ', '_')  # Replace spaces with underscores
    df_clean.columns = df_clean.columns.str.replace('[^A-Za-z0-9_]', '', regex=True)  # Remove special chars
    
    # 4. Handle missing values by data type
    numeric_columns = df_clean.select_dtypes(include=[np.number]).columns
    categorical_columns = df_clean.select_dtypes(include=['object']).columns
    
    print(f"Numeric columns: {len(numeric_columns)}")
    print(f"Categorical columns: {len(categorical_columns)}")
    
    # 5. Clean numeric columns
    for col in numeric_columns:
        # Replace infinite values with NaN
        df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
        
        # Handle outliers (values beyond 3 standard deviations)
        if df_clean[col].std() > 0:  # Only if there's variation
            mean_val = df_clean[col].mean()
            std_val = df_clean[col].std()
            outlier_threshold = 3 * std_val
            
            # Cap outliers instead of removing them
            df_clean[col] = np.where(
                df_clean[col] > mean_val + outlier_threshold,
                mean_val + outlier_threshold,
                df_clean[col]
            )
            df_clean[col] = np.where(
                df_clean[col] < mean_val - outlier_threshold,
                mean_val - outlier_threshold,
                df_clean[col]
            )
    
    # 6. Clean categorical columns
    for col in categorical_columns:
        # Strip whitespace and standardize case
        df_clean[col] = df_clean[col].astype(str).str.strip().str.title()
        
        # Replace common variations
        df_clean[col] = df_clean[col].replace({
            'Nan': np.nan,
            'None': np.nan,
            'N/A': np.nan,
            '': np.nan
        })
    
    print(f"Cleaned data shape: {df_clean.shape}")
    
    return df_clean


In [7]:
def handle_missing_values(df, strategy='smart'):
    """
    Handle missing values using different strategies
    
    Parameters:
    df: pandas DataFrame
    strategy: str - 'smart', 'median', 'mean', 'mode', 'drop'
    
    Returns:
    df_imputed: pandas DataFrame with missing values handled
    """
    df_imputed = df.copy()
    
    # Get missing value statistics
    missing_stats = df_imputed.isnull().sum()
    missing_percent = (missing_stats / len(df_imputed)) * 100
    
    print("Missing Value Analysis:")
    print("=" * 40)
    for col in missing_stats[missing_stats > 0].index:
        print(f"{col}: {missing_stats[col]} ({missing_percent[col]:.1f}%)")
    
    if strategy == 'smart':
        # Smart imputation based on data type and missing percentage
        for col in df_imputed.columns:
            missing_pct = missing_percent[col]
            
            if missing_pct > 70:
                # Drop columns with >70% missing data
                print(f"Dropping {col} - too many missing values ({missing_pct:.1f}%)")
                df_imputed = df_imputed.drop(columns=[col])
                
            elif missing_pct > 0:
                if df_imputed[col].dtype in ['int64', 'float64']:
                    # For numeric: use median (robust to outliers)
                    df_imputed[col] = df_imputed[col].fillna(df_imputed[col].median())
                    print(f"Filled {col} with median value")
                    
                else:
                    # For categorical: use mode or 'Unknown'
                    if not df_imputed[col].mode().empty:
                        df_imputed[col] = df_imputed[col].fillna(df_imputed[col].mode()[0])
                        print(f"Filled {col} with mode value")
                    else:
                        df_imputed[col] = df_imputed[col].fillna('Unknown')
                        print(f"Filled {col} with 'Unknown'")
    
    elif strategy == 'median':
        # Use median for numeric, mode for categorical
        numeric_cols = df_imputed.select_dtypes(include=[np.number]).columns
        categorical_cols = df_imputed.select_dtypes(include=['object']).columns
        
        for col in numeric_cols:
            df_imputed[col] = df_imputed[col].fillna(df_imputed[col].median())
        
        for col in categorical_cols:
            df_imputed[col] = df_imputed[col].fillna(df_imputed[col].mode()[0] if not df_imputed[col].mode().empty else 'Unknown')
    
    elif strategy == 'drop':
        # Drop rows with any missing values
        df_imputed = df_imputed.dropna()
        print(f"Dropped rows with missing values. New shape: {df_imputed.shape}")
    
    print(f"Final shape after handling missing values: {df_imputed.shape}")
    return df_imputed


In [8]:
def normalize_data(df, method='standard', exclude_columns=None):
    """
    Normalize/scale numerical data
    
    Parameters:
    df: pandas DataFrame
    method: str - 'standard', 'minmax', 'robust', 'quantile'
    exclude_columns: list - columns to exclude from scaling
    
    Returns:
    df_scaled: pandas DataFrame with scaled features
    scaler: fitted scaler object
    """
    df_scaled = df.copy()
    
    # Identify numeric columns
    numeric_columns = df_scaled.select_dtypes(include=[np.number]).columns.tolist()
    
    # Remove excluded columns
    if exclude_columns:
        numeric_columns = [col for col in numeric_columns if col not in exclude_columns]
    
    print(f"Scaling {len(numeric_columns)} numeric columns using {method} method")
    
    # Select appropriate scaler
    if method == 'standard':
        scaler = StandardScaler()
        print("Using StandardScaler (mean=0, std=1)")
        
    elif method == 'minmax':
        scaler = MinMaxScaler()
        print("Using MinMaxScaler (range 0-1)")
        
    elif method == 'robust':
        from sklearn.preprocessing import RobustScaler
        scaler = RobustScaler()
        print("Using RobustScaler (median-based, handles outliers)")
        
    elif method == 'quantile':
        from sklearn.preprocessing import QuantileTransformer
        scaler = QuantileTransformer(output_distribution='normal')
        print("Using QuantileTransformer (uniform to normal distribution)")
    
    # Fit and transform the data
    if numeric_columns:
        df_scaled[numeric_columns] = scaler.fit_transform(df_scaled[numeric_columns])
        
        # Display scaling statistics
        print("\\nScaling Statistics:")
        print("=" * 50)
        for col in numeric_columns[:5]:  # Show first 5 columns
            original_mean = df[col].mean()
            original_std = df[col].std()
            scaled_mean = df_scaled[col].mean()
            scaled_std = df_scaled[col].std()
            
            print(f"{col}:")
            print(f"  Original: mean={original_mean:.3f}, std={original_std:.3f}")
            print(f"  Scaled:   mean={scaled_mean:.3f}, std={scaled_std:.3f}")
        
        if len(numeric_columns) > 5:
            print(f"... and {len(numeric_columns)-5} more columns")
    
    return df_scaled, scaler


In [9]:
def encode_categorical_data(df, encoding_method='label', high_cardinality_threshold=10):
    """
    Encode categorical variables
    
    Parameters:
    df: pandas DataFrame
    encoding_method: str - 'label', 'onehot', 'target', 'binary'
    high_cardinality_threshold: int - threshold for high cardinality features
    
    Returns:
    df_encoded: pandas DataFrame with encoded features
    encoders: dict - fitted encoder objects
    """
    df_encoded = df.copy()
    encoders = {}
    
    # Identify categorical columns
    categorical_columns = df_encoded.select_dtypes(include=['object']).columns.tolist()
    
    print(f"Encoding {len(categorical_columns)} categorical columns")
    print(f"Method: {encoding_method}")
    
    for col in categorical_columns:
        unique_values = df_encoded[col].nunique()
        print(f"\\n{col}: {unique_values} unique values")
        
        if unique_values <= 1:
            # Drop columns with no variation
            print(f"  Dropping {col} - no variation")
            df_encoded = df_encoded.drop(columns=[col])
            continue
        
        if encoding_method == 'label':
            # Label encoding
            le = LabelEncoder()
            df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
            encoders[col] = le
            print(f"  Applied Label Encoding")
            
        elif encoding_method == 'onehot':
            if unique_values <= high_cardinality_threshold:
                # One-hot encoding for low cardinality
                dummies = pd.get_dummies(df_encoded[col], prefix=col)
                df_encoded = pd.concat([df_encoded, dummies], axis=1)
                df_encoded = df_encoded.drop(columns=[col])
                encoders[col] = list(dummies.columns)
                print(f"  Applied One-Hot Encoding ({len(dummies.columns)} new columns)")
            else:
                # Label encoding for high cardinality
                le = LabelEncoder()
                df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
                encoders[col] = le
                print(f"  Applied Label Encoding (high cardinality)")
        
        elif encoding_method == 'binary':
            # Binary encoding (good for medium cardinality)
            try:
                import category_encoders as ce
                be = ce.BinaryEncoder(cols=[col])
                encoded_cols = be.fit_transform(df_encoded[col])
                df_encoded = pd.concat([df_encoded, encoded_cols], axis=1)
                df_encoded = df_encoded.drop(columns=[col])
                encoders[col] = be
                print(f"  Applied Binary Encoding ({len(encoded_cols.columns)} new columns)")
            except ImportError:
                print(f"  Binary encoding not available, using Label Encoding")
                le = LabelEncoder()
                df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
                encoders[col] = le
    
    print(f"\\nFinal shape after encoding: {df_encoded.shape}")
    return df_encoded, encoders


In [10]:
def complete_preprocessing_pipeline(df, 
                                   clean_data=True,
                                   handle_missing='smart',
                                   normalize_method='standard',
                                   encoding_method='label',
                                   exclude_from_scaling=None):
    """
    Complete preprocessing pipeline that combines all steps
    
    Parameters:
    df: pandas DataFrame - raw data
    clean_data: bool - whether to clean the data
    handle_missing: str - missing value strategy
    normalize_method: str - scaling method
    encoding_method: str - categorical encoding method
    exclude_from_scaling: list - columns to exclude from scaling
    
    Returns:
    processed_df: pandas DataFrame - fully processed data
    preprocessing_info: dict - information about preprocessing steps
    """
    
    print("🚀 STARTING COMPLETE PREPROCESSING PIPELINE")
    print("=" * 60)
    
    processed_df = df.copy()
    preprocessing_info = {
        'original_shape': df.shape,
        'steps_applied': [],
        'encoders': {},
        'scaler': None
    }
    
    # Step 1: Data Cleaning
    if clean_data:
        print("\\n📋 STEP 1: DATA CLEANING")
        print("-" * 30)
        processed_df = clean_player_data(processed_df)
        preprocessing_info['steps_applied'].append('data_cleaning')
    
    # Step 2: Handle Missing Values
    if handle_missing:
        print("\\n🔧 STEP 2: HANDLING MISSING VALUES")
        print("-" * 30)
        processed_df = handle_missing_values(processed_df, strategy=handle_missing)
        preprocessing_info['steps_applied'].append(f'missing_values_{handle_missing}')
    
    # Step 3: Encode Categorical Variables
    if encoding_method:
        print("\\n🏷️ STEP 3: ENCODING CATEGORICAL VARIABLES")
        print("-" * 30)
        processed_df, encoders = encode_categorical_data(processed_df, encoding_method=encoding_method)
        preprocessing_info['encoders'] = encoders
        preprocessing_info['steps_applied'].append(f'encoding_{encoding_method}')
    
    # Step 4: Normalize/Scale Data
    if normalize_method:
        print("\\n📊 STEP 4: NORMALIZING DATA")
        print("-" * 30)
        processed_df, scaler = normalize_data(processed_df, 
                                            method=normalize_method,
                                            exclude_columns=exclude_from_scaling)
        preprocessing_info['scaler'] = scaler
        preprocessing_info['steps_applied'].append(f'scaling_{normalize_method}')
    
    # Final summary
    preprocessing_info['final_shape'] = processed_df.shape
    
    print("\\n✅ PREPROCESSING PIPELINE COMPLETED")
    print("=" * 60)
    print(f"Original shape: {preprocessing_info['original_shape']}")
    print(f"Final shape: {preprocessing_info['final_shape']}")
    print(f"Steps applied: {', '.join(preprocessing_info['steps_applied'])}")
    
    return processed_df, preprocessing_info


In [ ]:
# Example usage of the complete preprocessing pipeline
print("=== EXAMPLE: PREPROCESSING PIPELINE USAGE ===")

# Load sample data
sample_data = load_all_data("players.csv")

if sample_data:
    # Get the first dataset
    data_key = list(sample_data.keys())[0]
    raw_df = sample_data[data_key]
    
    print(f"\\nProcessing {data_key} dataset...")
    print(f"Original shape: {raw_df.shape}")
    
    # Apply complete preprocessing pipeline
    processed_df, info = complete_preprocessing_pipeline(
        raw_df,
        clean_data=True,
        handle_missing='smart',
        normalize_method='standard',
        encoding_method='label',
        exclude_from_scaling=['League', 'Team']  # Don't scale these if they exist
    )
    
    print(f"\\n📈 PREPROCESSING SUMMARY:")
    print(f"Shape change: {info['original_shape']} → {info['final_shape']}")
    print(f"Columns reduced/added: {info['original_shape'][1] - info['final_shape'][1]}")
    
    # Show sample of processed data
    print(f"\\n📋 SAMPLE OF PROCESSED DATA:")
    print(processed_df.head(3))
    
else:
    print("No sample data loaded. Please check your data directory.")


=== EXAMPLE: PREPROCESSING PIPELINE USAGE ===
Loading players.csv...
  ✓ Loaded 5942 records from 157 teams
\nProcessing players dataset...
Original shape: (5942, 38)
🚀 STARTING COMPLETE PREPROCESSING PIPELINE
\n📋 STEP 1: DATA CLEANING
------------------------------
Original data shape: (5942, 38)
Numeric columns: 30
Categorical columns: 8
Cleaned data shape: (5942, 38)
\n🔧 STEP 2: HANDLING MISSING VALUES
------------------------------
Missing Value Analysis:
Playing_Time_MP: 4916 (82.7%)
MP: 1026 (17.3%)
Dropping Playing_Time_MP - too many missing values (82.7%)
Filled MP with median value
Final shape after handling missing values: (5942, 37)
\n🏷️ STEP 3: ENCODING CATEGORICAL VARIABLES
------------------------------
Encoding 8 categorical columns
Method: label
\nPlayer: 5192 unique values
  Applied Label Encoding
\nNation: 120 unique values
  Applied Label Encoding
\nPos: 11 unique values
  Applied Label Encoding
\nAge: 1310 unique values
  Applied Label Encoding
\nMatches: 2 unique v

: 